In [23]:
import pandas as pd

In [24]:
# Import the dataframe

df_fifa_raw = pd.read_csv("fifa21_raw_data.csv", low_memory=False, encoding="utf-8")

# Visualize the file

df_fifa_raw.head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,...,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,...,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,...,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,...,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,...,High,Medium,5 ★,91,85,86,94,36,59,\n273


In [ ]:
# Checking data types

df_fifa_raw.info()

In [ ]:
# I'll start by separately treating the "Object" type columns on which we'll do an initial cleaning of whitespace, line breaks, and possible double spaces.

object_columns = []

for column in df_fifa_raw.columns:
    if pd.api.types.is_object_dtype(df_fifa_raw[column]):
        object_columns.append(column)    
    df_fifa_raw[column] = df_fifa_raw[column].astype("str").str.strip()
    df_fifa_raw[column] = df_fifa_raw[column].astype("str").str.replace("\n", "")
    df_fifa_raw[column] = df_fifa_raw[column].astype("str").str.replace("  ", " ")

object_columns

In [6]:
# I'll extract the contract start and end years from the "Team & Contract" column to the "Contract From" and "Contract To" columns.


df_fifa_raw['Contract_From'] = df_fifa_raw['Team & Contract'].str.extract(r'(\d{4})\s*~')
df_fifa_raw['Contract_From'] = df_fifa_raw['Contract_From'].str.strip()
df_fifa_raw['Contract_From'] = df_fifa_raw['Contract_From'].str.replace("~", "")
df_fifa_raw['Contract_To'] = df_fifa_raw['Team & Contract'].str.extract(r'~\s*(\d{4})')
df_fifa_raw['Contract_To'] = df_fifa_raw['Contract_To'].str.strip()
df_fifa_raw['Contract_To'] = df_fifa_raw['Contract_To'].str.replace("~", "")

In [7]:
# In cases where there's a "loan" 

# 1. Extract the values

df_fifa_raw['Loan_From'] = df_fifa_raw.loc[df_fifa_raw["Team & Contract"].str.contains("Loan"), "Team & Contract"].str.extract(r'(\d{4})')
df_fifa_raw['Loan_To'] = df_fifa_raw.loc[df_fifa_raw["Team & Contract"].str.contains("Loan"), "Team & Contract"].str.extract(r'(\d{4})')

# 2. Replace the values in the Contract_From and Contract_To columns

df_fifa_raw['Contract_From'] = df_fifa_raw['Contract_From'].fillna(df_fifa_raw['Loan_From'])
df_fifa_raw['Contract_To'] = df_fifa_raw['Contract_To'].fillna(df_fifa_raw['Loan_To'])

In [8]:
# Remove the Loan_From and Loan_To columns. We take advantage of this step to remove the "Loan Date End" column as it is redundant considering the previous steps and most of its values are "nan"

df_fifa_raw.drop(columns=["Loan_From", "Loan_To", "Loan Date End"], inplace=True, axis=1)
df_fifa_raw.drop(columns=["Loan_From", "Loan_To", "Loan Date End"], inplace=True, axis=1)

In [ ]:
# Checking the resulting values of the extraction
df_fifa_raw[["Team & Contract", "Contract_From", "Contract_To"]].info()

In [ ]:
# There are still some empty values and this is because there are players who do not have a current contract, i.e., they are free agents

df_fifa_raw[(df_fifa_raw["Contract_From"].isnull()) & (df_fifa_raw["Contract_To"].isnull())][["Team & Contract","Contract_From", "Contract_To"]].head()

In [ ]:
# 238 records will be removed.

df_fifa_raw.dropna(inplace=True, axis=0)

df_fifa_raw.info()

In [13]:
# Removing the year and the "~" character from the "Team & Contract" column
df_fifa_raw['Team & Contract'] = df_fifa_raw['Team & Contract'].str.replace(r'(\d{4})\s*~\s*(\d{4})', '', regex=True).str.strip()
df_fifa_raw["Team & Contract"] = df_fifa_raw["Team & Contract"].str.replace(r'1. ', "", regex=True)

# Also removing the loan dates

df_fifa_raw['Team & Contract'] = df_fifa_raw['Team & Contract'].str.replace(r' On Loan', "").str.strip()

loan_dates = ['Nov 22, 2020','Nov 30, 2020','Dec 30, 2020','Dec 31, 2020','Jan 1, 2021','Jan 3, 2021','Jan 4, 2021','Jan 17, 2021','Jan 30, 2021','Jan 31, 2021','May 31, 2021','Jun 23, 2021','Jun 30, 2021',"Jul 1, 2021","Jul 5, 2021","Jul 31, 2021",'Jan 31, 2021','Aug 1, 2021','Aug 31, 2021',"Dec 30, 2021","Dec 31, 2021",'Nov 27, 2021','May 31, 2022','Jun 30, 2022','Jun 30, 2023']

for loan_date in loan_dates:
    
    df_fifa_raw['Team & Contract'] = df_fifa_raw['Team & Contract'].str.replace(loan_date, "").str.strip()



In [15]:
# Rename the column as "Team"
df_fifa_raw.rename(columns={"Team & Contract": "Team"}, inplace=True)

In [16]:
# Removing the "★" character from the "IR", "SM" and "WF" columns

df_fifa_raw["IR"] = df_fifa_raw["IR"].str.replace("★", "")
df_fifa_raw["SM"] = df_fifa_raw["SM"].str.replace("★", "")
df_fifa_raw["W/F"] = df_fifa_raw["W/F"].str.replace("★", "")

In [17]:
   
# Removing the "€", "K" and "M" symbols from the "Value", "Wage", "Hits" and "Release Clause" columns. We'll convert these columns to numeric type later by making the corresponding conversion.

for i in ["Value","Wage","Hits","Release Clause"]:
    df_fifa_raw[i] = df_fifa_raw[i].str.replace("€", "")
    df_fifa_raw[i] = df_fifa_raw[i].str.replace("M", "")
    df_fifa_raw[i] = df_fifa_raw[i].str.replace("K", "")   


# Remove the "lbs" unit from the "Weight" column to be able to convert that column to numeric type

df_fifa_raw["Weight"] = df_fifa_raw["Weight"].str.replace("lbs", "")

In [ ]:
# Standardizing column names by replacing some of them

df_fifa_raw.rename({"photoUrl": "PhotoUrl", "playerUrl": "PlayerUrl", "foot":"Foot", "↓OVA": "OVA" }, inplace= True, axis=1)

columns = df_fifa_raw.columns.to_series()
columns = columns.str.replace(" ", "_")

df_fifa_raw.columns = columns

df_fifa_raw.info()

# Reordering all columns alphabetically

df_fifa_raw.sort_index(inplace=True, axis=1)

In [ ]:
# We repeat once again the process of selecting the columns that are of type "object", as now we will change some of them to int type and to date type

object_columns = []

for column in df_fifa_raw.columns:    
    if pd.api.types.is_object_dtype(df_fifa_raw[column]):
        object_columns.append(column)

object_columns


In [21]:
# Changing the "Joined" column to datetime type

df_fifa_raw["Joined"] = df_fifa_raw["Joined"].astype("datetime64[s]")


In [22]:
# Chaing several columns to int type

to_int_list = [
    "Acceleration", "Age", "Aggression", "Agility", "Attacking", "BOV", "Balance", "Ball_Control", "Base_Stats", 
    "Composure", "Contract_From", "Contract_To", "Crossing", "Curve", "DEF", "DRI", "Defending", "Dribbling", 
    "FK_Accuracy", "Finishing", "GK_Diving", "GK_Handling", "GK_Kicking", "GK_Positioning", "GK_Reflexes", 
    "Goalkeeping", "Growth", "Heading_Accuracy","IR", "Interceptions", "Jumping", "Long_Passing", 
    "Long_Shots", "Marking", "Mentality", "Movement", "OVA", "PAC", "PAS", "PHY", "POT", "Penalties", 
    "Positioning", "Power", "Reactions", "SHO", "SM", "Short_Passing", "Shot_Power", "Skill", "Sliding_Tackle", 
    "Sprint_Speed", "Stamina", "Standing_Tackle", "Strength", "Total_Stats", "Vision", "Volleys", "W/F","Weight"
]

for n in to_int_list:
    df_fifa_raw[n] = df_fifa_raw[n].astype('int64')

for i in ["Value", "Wage","Hits","Release_Clause"]:
    df_fifa_raw[i] = df_fifa_raw[i].astype('Float64')

# and float:

for i in ["Value", "Wage","Hits","Release_Clause"]:
    df_fifa_raw[i] = df_fifa_raw[i].astype('Float64')

In [24]:
# Calculating correct values for the "Value", "Wage", "Hits" and "Release_Clause" columns


# Value and Release Clause x 1000000
for a in ["Value", "Release_Clause"]:
    df_fifa_raw[a] = df_fifa_raw[a].apply(lambda x: x*1000000)

# Wage and Hits x 1000
for a in ["Wage", "Hits"]:
    df_fifa_raw[a] = df_fifa_raw[a].apply(lambda x: x*1000)

In [ ]:
# Final check
df_fifa_raw.info()

In [27]:
# exporting the clean csv with no idex column

df_fifa_raw.to_csv("fifa21_clean.csv", index=None)